In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 65.0 MB/s eta 0:00:00:00:0100:01


In [3]:
import duckdb as ddb
import pandas as pd

In [4]:
df=pd.read_excel("/kaggle/input/movie-dataset-imdb/imdb_top_1000 (1).xlsx")
df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Actual Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142.0,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110.0,2.834147e+07,2.834147e+07
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175.0,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367.0,1.349664e+08,1.349664e+08
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152.0,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232.0,5.348584e+08,5.348584e+08
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202.0,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952.0,5.730000e+07,5.730000e+07
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96.0,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845.0,4.360000e+06,4.360000e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118.0,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374.0,3.050000e+07,3.050000e+07
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97.0,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471.0,NaN,6.803475e+07
999,https://m.media-amazon.com/images/M/MV5BMTY5OD...,The 39 Steps,1935,NaN,86.0,"Crime, Mystery, Thriller",7.6,A man in London tries to help a counter-espion...,93.0,Alfred Hitchcock,Robert Donat,Madeleine Carroll,Lucie Mannheim,Godfrey Tearle,51853.0,NaN,6.803475e+07
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.dtypes

Poster_Link       object
Series_Title      object
Released_Year     object
Certificate       object
Runtime          float64
Genre             object
IMDB_Rating      float64
Overview          object
Meta_score       float64
Director          object
Star1             object
Star2             object
Star3             object
Star4             object
No_of_Votes      float64
Gross            float64
Actual Gross     float64
dtype: object

In [6]:
ddb.sql("""CREATE TABLE df_copy AS
SELECT * FROM df;

DELETE FROM df_copy
WHERE Director IS NULL""")

In [7]:
ddb.sql("SELECT * FROM df_copy")

┌──────────────────────┬──────────────────────┬───────────────┬───┬─────────────┬─────────────┬──────────────┐
│     Poster_Link      │     Series_Title     │ Released_Year │ … │ No_of_Votes │    Gross    │ Actual Gross │
│       varchar        │       varchar        │    varchar    │   │   double    │   double    │    double    │
├──────────────────────┼──────────────────────┼───────────────┼───┼─────────────┼─────────────┼──────────────┤
│ https://m.media-am…  │ The Shawshank Rede…  │ 1994          │ … │   2343110.0 │  28341469.0 │   28341469.0 │
│ https://m.media-am…  │ The Godfather        │ 1972          │ … │   1620367.0 │ 134966411.0 │  134966411.0 │
│ https://m.media-am…  │ The Dark Knight      │ 2008          │ … │   2303232.0 │ 534858444.0 │  534858444.0 │
│ https://m.media-am…  │ The Godfather: Par…  │ 1974          │ … │   1129952.0 │  57300000.0 │   57300000.0 │
│ https://m.media-am…  │ 12 Angry Men         │ 1957          │ … │    689845.0 │   4360000.0 │    4360000.0 │
│

In [8]:
# 1. Director's Impact on Earnings: Analyze how movies directed by different directors perform in terms of gross earnings. Are there any noticeable trends or patterns?


ddb.sql("""SELECT 
                Director,count(Director) AS No_of_Movies,round(Avg("Actual Gross"),2) AS AvgGrossEarnings,
                Max("Actual Gross") AS HighestGross 
                FROM df_copy
                Group by Director, Order by AvgGrossEarnings Desc
""")

┌──────────────────────┬──────────────┬──────────────────┬──────────────┐
│       Director       │ No_of_Movies │ AvgGrossEarnings │ HighestGross │
│       varchar        │    int64     │      double      │    double    │
├──────────────────────┼──────────────┼──────────────────┼──────────────┤
│ Anthony Russo        │            4 │     551259850.75 │  858373000.0 │
│ Gareth Edwards       │            1 │      532177324.0 │  532177324.0 │
│ J.J. Abrams          │            3 │     474390301.67 │  936662225.0 │
│ Josh Cooley          │            1 │      434038008.0 │  434038008.0 │
│ Roger Allers         │            1 │      422783777.0 │  422783777.0 │
│ Tim Miller           │            1 │      363070709.0 │  363070709.0 │
│ James Gunn           │            2 │      361494850.5 │  389813101.0 │
│ James Cameron        │            5 │      349647320.4 │  760507625.0 │
│ Byron Howard         │            1 │      341268248.0 │  341268248.0 │
│ David Yates          │            3 

In [9]:
# 2. Genre Popularity Over the Years: Investigate how different genres have evolved in popularity over time.

ddb.sql("""SELECT Genre,Released_Year, Avg(IMDB_Rating) AS AvgRating
            FROM df_copy
            Group By Genre,Released_Year
            Order By Genre,Released_Year""")

┌──────────────────────────────┬───────────────┬───────────┐
│            Genre             │ Released_Year │ AvgRating │
│           varchar            │    varchar    │  double   │
├──────────────────────────────┼───────────────┼───────────┤
│ Action, Adventure            │ 1981          │       8.4 │
│ Action, Adventure            │ 1982          │       7.7 │
│ Action, Adventure            │ 1989          │       8.2 │
│ Action, Adventure            │ 2005          │       8.2 │
│ Action, Adventure            │ 2012          │       8.4 │
│ Action, Adventure, Biography │ 1972          │       7.9 │
│ Action, Adventure, Comedy    │ 1926          │       8.1 │
│ Action, Adventure, Comedy    │ 1975          │       8.2 │
│ Action, Adventure, Comedy    │ 1980          │       7.9 │
│ Action, Adventure, Comedy    │ 2014          │      7.85 │
│         ·                    │  ·            │        ·  │
│         ·                    │  ·            │        ·  │
│         ·             

In [9]:
# 3. Correlation Between IMDB Ratings and Commercial Success: Explore if there's a relationship between a movie's IMDB rating and its box office earnings.


ddb.sql("""SELECT round(corr(IMDB_Rating,"Actual Gross"),2) as Correlation_value,
        CASE 
           WHEN round(corr(IMDB_Rating,"Actual Gross"),2)>0 and round(corr(IMDB_Rating,Gross),2)<1 then 'RatingIncreases then GrossIncreases'
           WHEN round(corr(IMDB_Rating,"Actual Gross"),2)<0 then 'RatingIncreases then GrossDecreases'
           WHEN round(corr(IMDB_Rating,"Actual Gross"),2)=0 then 'NoImpact'
           
        END  AS Result from df_copy
       """)

┌───────────────────┬─────────────────────────────────────┐
│ Correlation_value │               Result                │
│      double       │               varchar               │
├───────────────────┼─────────────────────────────────────┤
│              0.09 │ RatingIncreases then GrossIncreases │
└───────────────────┴─────────────────────────────────────┘

In [10]:
# Impact of Movie lenght on its ratings and success

ddb.sql(""" SELECT Runtime, round(Avg(IMDB_Rating),2) AS AvgRating,
            round(Avg("Actual Gross"),2) AS AvgGross
            From df_copy
            Group by Runtime, Order by AvgGross
""")

┌─────────┬───────────┬──────────────┐
│ Runtime │ AvgRating │   AvgGross   │
│ double  │  double   │    double    │
├─────────┼───────────┼──────────────┤
│   224.0 │       8.1 │      70147.0 │
│   205.0 │       8.1 │     102021.0 │
│   196.0 │       8.1 │     165520.0 │
│   207.0 │       8.6 │     269061.0 │
│    45.0 │       8.2 │     977375.0 │
│    67.0 │       8.1 │    1033895.0 │
│   210.0 │       8.2 │    1223240.0 │
│   186.0 │       8.1 │    1706833.5 │
│   184.0 │       7.8 │    1782795.0 │
│    75.0 │       7.9 │    2205485.0 │
│      ·  │        ·  │        ·     │
│      ·  │        ·  │        ·     │
│      ·  │        ·  │        ·     │
│   169.0 │      8.27 │ 176955905.25 │
│   143.0 │      8.02 │ 178788551.87 │
│   162.0 │      7.88 │  187810791.8 │
│   238.0 │       8.1 │  198676459.0 │
│   179.0 │      8.45 │ 205293057.94 │
│   201.0 │      8.25 │ 222940327.94 │
│   149.0 │      8.28 │ 228756334.75 │
│   164.0 │       8.2 │  270096629.0 │
│   181.0 │      8.13 │  

In [15]:
# 4. Impact of Movie Length on Ratings or Earnings: Examine whether the duration of a movie influences its ratings or financial success.

ddb.sql(""" SELECT round(corr(Runtime,IMDB_Rating),2) as Correlation_value,
        CASE 
           WHEN round(corr(Runtime,IMDB_Rating),2)>0 and round(corr(Runtime,IMDB_Rating),2)<1 then 'RuntimeIncreases then RatingIncreases'
           WHEN round(corr(Runtime,IMDB_Rating),2)<0 then 'RuntimeIncreases then RatingDecreases'
           WHEN round(corr(Runtime,IMDB_Rating),2)=0 then 'NoImpact'
           
        END  AS Result from df_copy
       """)




┌───────────────────┬───────────────────────────────────────┐
│ Correlation_value │                Result                 │
│      double       │                varchar                │
├───────────────────┼───────────────────────────────────────┤
│              0.24 │ RuntimeIncreases then RatingIncreases │
└───────────────────┴───────────────────────────────────────┘

In [16]:
ddb.sql(""" SELECT round(corr(Runtime,"Actual Gross"),2) as Correlation_value,
        CASE 
           WHEN round(corr(Runtime,"Actual Gross"),2)>0 and round(corr(Runtime,"Actual Gross"),2)<1 then 'RuntimeIncreases then GrossIncreases'
           WHEN round(corr(Runtime,"Actual Gross"),2)<0 then 'RuntimeIncreases then GrossDecreases'
           WHEN round(corr(Runtime,"Actual Gross"),2)=0 then 'NoImpact'
           
        END  AS Result from df_copy
       """)

┌───────────────────┬──────────────────────────────────────┐
│ Correlation_value │                Result                │
│      double       │               varchar                │
├───────────────────┼──────────────────────────────────────┤
│              0.12 │ RuntimeIncreases then GrossIncreases │
└───────────────────┴──────────────────────────────────────┘

In [14]:
# 5. Actor Influence on Movie Success: Analyze how the presence of certain actors correlates with a movie's performance in terms of ratings and earnings.

ddb.sql(""" SELECT Star1,Avg(IMDB_Rating) AS Avg_Rating,sum("Actual Gross") AS Total_Gross
            From df_copy 
            Group by Star1 
            Order by Total_Gross Desc
            
""")

┌───────────────────┬───────────────────┬──────────────┐
│       Star1       │    Avg_Rating     │ Total_Gross  │
│      varchar      │      double       │    double    │
├───────────────────┼───────────────────┼──────────────┤
│ Tom Hanks         │ 8.041666666666666 │ 2493097454.0 │
│ Joe Russo         │ 8.075000000000001 │ 2205039403.0 │
│ Leonardo DiCaprio │ 8.133333333333333 │ 1877321752.0 │
│ Daniel Radcliffe  │ 7.766666666666667 │ 1835901034.0 │
│ Christian Bale    │ 8.112499999999999 │ 1351591432.0 │
│ Robert Downey Jr. │ 7.833333333333333 │ 1150720327.0 │
│ Elijah Wood       │               8.8 │ 1035942020.0 │
│ Daisy Ridley      │               7.9 │  936662225.0 │
│ Mark Hamill       │ 8.533333333333333 │  922340616.0 │
│ Craig T. Nelson   │               7.8 │  870022836.0 │
│        ·          │                ·  │         ·    │
│        ·          │                ·  │         ·    │
│        ·          │                ·  │         ·    │
│ Aleksandr Antonov │          

In [21]:
# 6. Release Date Analysis: Study the impact of a movie's release date (e.g., month, season) on its success.

ddb.sql(""" SELECT Released_Year, round(sum("Actual Gross"),2) AS TotalGrossInTheYear
            From df_copy
            Group by Released_Year, Order by TotalGrossInTheYear Desc
        """)


┌───────────────┬─────────────────────┐
│ Released_Year │ TotalGrossInTheYear │
│    varchar    │       double        │
├───────────────┼─────────────────────┤
│ 2009          │       3209309588.48 │
│ 2018          │       2947931116.35 │
│ 2016          │       2935731179.35 │
│ 2019          │       2882985944.09 │
│ 2014          │       2823663971.87 │
│ 2012          │       2746720289.61 │
│ 2015          │       2598406369.74 │
│ 2010          │       2558256632.87 │
│ 2001          │       2233888419.74 │
│ 2017          │       2197382353.74 │
│  ·            │               ·     │
│  ·            │               ·     │
│  ·            │               ·     │
│ 1952          │          28874268.0 │
│ 1925          │           5500970.0 │
│ 1921          │           5450000.0 │
│ 1941          │           3693694.0 │
│ 1930          │           3270000.0 │
│ 1927          │           1775706.0 │
│ 1926          │           1033895.0 │
│ 1924          │            977375.0 │


In [22]:
ddb.sql("""CREATE TABLE df_copy2 AS
SELECT * FROM df_copy;

Update df_copy2 
SET Released_Year=1994
WHERE Released_Year='PG'
""")

In [25]:
# 6. Release Date Analysis: Study the impact of a movie's release date (e.g., month, season) on its success.


ddb.sql(""" 
    SELECT 
        round(corr(CAST(Released_Year AS INT), "Actual Gross"), 2) as Correlation_value,
        CASE 
           WHEN round(corr(CAST(Released_Year AS INT), "Actual Gross"), 2) > 0 AND round(corr(CAST(Released_Year AS INT), "Actual Gross"), 2) < 1 
               THEN 'As release years advance, gross value experiences growth'
           WHEN round(corr(CAST(Released_Year AS INT), "Actual Gross"), 2) < 0 
               THEN 'An increase in release year correlates with a decrease in gross value'
           WHEN round(corr(CAST(Released_Year AS INT), "Actual Gross"), 2) = 0 
               THEN 'No Impact'
        END AS Result 
    FROM df_copy2
""")


┌───────────────────┬──────────────────────────────────────────────────────────┐
│ Correlation_value │                          Result                          │
│      double       │                         varchar                          │
├───────────────────┼──────────────────────────────────────────────────────────┤
│              0.19 │ As release years advance, gross value experiences growth │
└───────────────────┴──────────────────────────────────────────────────────────┘